![Callysto.ca Banner](https://github.com/callysto/curriculum-notebooks/blob/master/callysto-notebook-banner-top.jpg?raw=true)
 
<a href="https://hub.callysto.ca/jupyter/hub/user-redirect/git-pull?repo=https%3A%2F%2Fgithub.com%2Fcallysto%2FML-exploration&branch=main&urlpath=image-classification/01-blurring-and-sharpness.ipynb&depth=1" target="_parent"><img src="https://raw.githubusercontent.com/callysto/curriculum-notebooks/master/open-in-callysto-button.svg?sanitize=true" width="123" height="24" alt="Open in Callysto"></a>

# Image Processing

In the previous notebook, we went over the details of how images are represented as arrays of numbers. The **pixel** is the smallest unit of measurement in digital art, and the pixel's colour is represented numerically by mixing different amounts of colour (i.e. RGB, RGBA) or black and white (i.e. grayscale). Information in the header of the image file will describe the format and the dimensions of the image, allowing the final picture to be drawn by software.

Capturing, storing, and retrieving the image files is just a matter of reading this information. But what if we want to manipulate the images? For example, how does software like Photoshop, Microsoft Paint, or others change the values of the underlying numbers to produce the effects the user wants? In this notebook, we'll investigate how different techniques are implemented at the data level, and allow you to interact with the tools to modify the image.

## Blurring

As we talked about in the previous notebook, resolution is an important feature of an image when it comes to the detail available. Pictures with lower resolutions have fewer pixels to cover the same physical size, and therefore contain less information about what they're trying to show. If you have the same image, one with a higher resolution and one with a lower, and make them the same physical size, the difference in quality is apparent:

<center><img src="https://upload.wikimedia.org/wikipedia/commons/f/f2/Resolution_illustration.png" /></center>
<center> <a href="https://en.wikipedia.org/wiki/Image_resolution">https://en.wikipedia.org/wiki/Image_resolution </a></center><br>



Blurring has a similar effect on an image: reducing the detail and making the edges less sharp. That's simple to see in the image above, as we move from right to left across the different resolutions. Lower resolution photos can be easily represented in higher resolutions (i.e. a 1x1 pixel in 50x50 would be a solid colour 2x2 pixel in 100x100), but how does blurring affect the underlying numbers? There are [many reasons why](https://www.photographygoals.com/blur-background-in-photos/) blurring might be used as a stylistic choice, so we'll dive into a few of the different types of blurring.

We'll import *A Sunday Afternoon on the Island of La Grande Jatte* by Georges Seurat, a painting with significant line detail, to really highlight the effect of each filter. The image is high resolution (1280x861)

In [ ]:
from PIL import Image, ImageFilter
import matplotlib.pyplot as plt
import numpy as np
import cv2
painting = Image.open('img/A_Sunday_on_La_Grande_Jatte.jpg')
display(painting)

### Kernels

Before we jump into the techniques used by image editing software, it's important to understand *how* the manipiulations are performed. For most techniques (and all techniques in this notebook), the editing consists of a *filter* that "slides" across the image and modifies the values of each pixel according to the values inside the [kernel](https://en.wikipedia.org/wiki/Kernel_(image_processing)). The kernel is a matrix of values, centered on the pixel whose value is being modified, and the final value of that pixel is the product of the filter matrix and the pixels bounded by the kernel. 

For example, below is a 5x5 kernel of an *identity matrix*, which is a filter that would result in no change to the pixel and surrounding pixels. Applied to any 5x5 pixel section in an image, the center pixel would retain its original value and the outlying pixels would not have any input:

```
[0, 0, 0, 0, 0]
[0, 0, 0, 0, 0]
[0, 0, 1, 0, 0]
[0, 0, 0, 0, 0]
[0, 0, 0, 0, 0]
```


As we go over various filters, keep in mind that the kernel size is often a user-selectable value. The actual changes to the underlying pixel values are determined by the values inside the filter matrix.

### Mean Filter

This is the simplest type of blurring, employing a kernel size selected by the user, and modifying each pixel to be the arithmetic mean of all the values within the kernel. For example, a 3x3 kernel would slide across each pixel (with the pixel at the center), and set the value of the pixel to be the mean of the 9 values inside the kernel, with each pixel **weighted equally**. In colour images, this is done independently for each colour channel, and the resulting mixed colour is used.

For this kernel (and nearly all kernels), the size must be an odd number, so that the pixel to be modified lays in the center of the kernel:

```
[1, 1, 1]
[1, 1, 1]
[1, 1, 1]
```

In this filter, the resulting value of the center pixel is the **average of the element-wise products of the filter and the kernel**. In a mean filter, the value of each of the elements in the kernel is the same, so a simple mean of the kernel's contents will have the same result. As we'll see later though with other kernels, it's helpful to think of them as averages of products.

In [ ]:
# Kernel
a1 = np.array([[1, 2, 6],
               [3, 5, 3],
               [7, 8, 2]])

# Filter
f1 = np.array([[1, 1, 1],
               [1, 1, 1],
               [1, 1, 1]])

result1 = a1 * f1
print(f'Product of mean filter and kernel:\n{result1}')

Applying this to the center pixel:

In [ ]:
pixel1 = np.mean(result1)
print('Center pixel value:', round(pixel1,0))

Try changing the width of the kernel below and see how the image is affected.

In [ ]:
## Mean filter
# Set 'w' to determine the width of the kernel:

w1 = 3

###########
if w1 % 2 == 0:
    print('Kernel size must be an odd number!')
else:
    meanFilter = painting.filter(ImageFilter.BoxBlur(radius=((abs(w1)-1)/2)))
    display(meanFilter)

### Gaussian Filter

One of the drawbacks of a mean filter is that all the pixels inside the kernel have the same weight when contributing to the averaged pixel value. Especially in large kernels, pixels that are far from the center are likely less related to the center pixel, yet they have the same effect as pixels adjacent to the center.

Gaussian filters attempt to limit this effect by weighting the pixels less when they're further away from the center. This allows nearer pixels to have a more potent impact on the filtered value, where the weights on the outlying pixels follow a *normal* (or *Gaussian*) distribution. In practice, Gaussian filters typically preserve line detail much better than mean filters. In the below example, the center pixel has a weight of 4, whereas the diagonal pixels have a weight of just 1:

```
[1, 2, 1]
[2, 4, 2]
[1, 2, 1]
```

Similar to the mean filter, the Gaussian filter is the average of the products of the kernel and the filter:

In [ ]:
# Gaussian filter
f2 = np.array([[1, 2, 1],
               [2, 4, 2],
               [1, 2, 1]])

# Element-wise multiplication with same kernel as mean filter
result2 = a1 * f2
print(f'Original kernel:\n{a1}\n')
print(f'Product of Gaussian filter and kernel:\n{result2}\n')

Applying this to the center pixel:

In [ ]:
pixel2 = np.mean(result2)
print('Center pixel value:', round(pixel2,0))

As the dropoff away from the center pixel follows a normal distribution, this kernel has an additional parameter of the *standard deviation* of the distribution. Try changing both the standard deviation (*sigma*) value and the kernel size to compare with the mean filter:

In [ ]:
## Gaussian Filter
# Change the kernel width and sigma value to see the effect on the image:

w2 = 5
sigma2 = 5

###########
if w2 % 2 == 0:
    print('Kernel size must be an odd number!')
else:
    gaussianFilter = Image.fromarray(cv2.GaussianBlur(np.array(painting), (abs(w2), abs(w2)), abs(sigma2)))
    display(gaussianFilter)

### Median Filter

As with any sampled data, pixel distributions that are described using their mean value can be quite sensitive to outliers, which the Gaussian filter limits, but does not eliminate. For all the same reasons it can be effective in a larger numerical dataset, utlizing the *median* value can be even more resistant to outliers. Instead of taking the mean of all the values in the kernel, the median filter takes, well, the median value. This has the effect of making sharp contrasts in colour stand out even more.

Try it below to see how different kernel sizes compare when using the median filter, versus the mean and Gaussian filters:

In [ ]:
## Median filter
# Set 'w' to determine the width of the kernel:

w3 = 3

###########
if w3 % 2 == 0:
    print('Kernel size must be an odd number!')
else:
    medianFilter = painting.filter(ImageFilter.MedianFilter(abs(w3)))
    display(medianFilter)

## Sharpness

Though there are artistic reasons why you might want to use blurring in an image, more commonly blurring is an undesirable artifact introduced when capturing the image. As we've seen above, it's simple to reduce the detail in an already detailed image, but it's much more difficult to take a blurry image and *increase* the detail. Nevertheless, clever techniques exist that can attempt to introduce clarity into images that lack it to begin with.

The technique we'll look at here is referred to as [**Unsharp Masking**](https://en.wikipedia.org/wiki/Unsharp_masking). Though it has its roots in darkroom photography, it can be simply applied to digital images.

Unsharping works by first applying a Gaussian blur, and then comparing the blurred image to the original. The difference between the two images (subject to some parameters we'll discuss in a second) is then added back to the original image to highlight the edges:

In [ ]:
# Convert image of painting to its pixel values
paintingArr = np.array(painting)
paintingArr

In the next step, we'll run a Gassian filter on the painting and subtract that blurred image from the original. Below is both the resulting array and the image the array represents:

In [ ]:
# Apply Gaussian filter (width=25, sigma=5) and convert resulting image to array; subtract from original
paintingGauss = Image.fromarray(cv2.GaussianBlur(np.array(painting), (25,25), 5))
paintingDiffArr = np.array(painting) - np.array(paintingGauss)
display(paintingDiffArr)
display(Image.fromarray(paintingDiffArr))

The new image is fairly noisy, but there are clearly some edges present that line up with colour transitions from the main painting. By smoothing the original image with the Gaussian filter, any sharp colour transitions (i.e. edges) that existed have been lessened by the filter, but still exist, and create the largest differences between the two images. 

By effectively highlighting the regions of an image most affected by blurring, and emphasizing those regions, an artificial sharpness can be added where previously the image would have been blurry:

In [ ]:
sharp = painting.filter(ImageFilter.UnsharpMask(25, 100, 0))
display(sharp)

[![Callysto.ca License](https://github.com/callysto/curriculum-notebooks/blob/master/callysto-notebook-banner-bottom.jpg?raw=true)](https://github.com/callysto/curriculum-notebooks/blob/master/LICENSE.md)